<a href="https://colab.research.google.com/github/fzanart/Socialz/blob/main/es_plus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scipy --upgrade

In [ ]:
import numpy as np
import pandas as pd
import json
import networkx as nx
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive/')
import random
from scipy.stats import qmc
from tqdm import tqdm
import time

In [ ]:
# Read the data
df = pd.read_csv('/content/drive/MyDrive/Socialz/cobol_pullrequests.csv', usecols=['login', 'name'])
df

In [ ]:
# Add user (u: ) repo (r: ) indicator for each node
df['login'] = df['login'].apply(lambda x: 'u: '+x)
df['name'] = df['name'].apply(lambda x: 'r: '+x)

In [ ]:
# Build the adjacency matrix for user - repo (and repo - user) interactions.
adj_matrix = pd.crosstab(df['login'], df['name']).astype(float)
idx = adj_matrix.columns.union(adj_matrix.index)
adj_matrix = adj_matrix.reindex(index = idx, columns=idx, fill_value=0.0)

In [ ]:
# Add user - user interactions.
matrix_elements = np.sort(df['login'].unique(), axis=0)
repo_len = len(df['name'].unique())

def  cosine_similarity(vector_a, vector_b):
    
    norma = (np.linalg.norm(vector_a)*np.linalg.norm(vector_b))

    if norma > 0:

        cos_sim = np.dot(vector_a,vector_b)/ norma

    else:

        cos_sim = 0

    return cos_sim

#Initialise an auxiliar matrix to record the user - user similarities
shape = (len(matrix_elements), len(matrix_elements))
aux_matrix = pd.DataFrame((np.zeros(shape=shape)), columns=matrix_elements, index=matrix_elements)

for i in matrix_elements:
    for j in matrix_elements:

        # Compare row vectors
        aux_vector_a = adj_matrix.loc[i].to_numpy()[:repo_len]
        aux_vector_b = adj_matrix.loc[j].to_numpy()[:repo_len]
        
        score = cosine_similarity(aux_vector_a, aux_vector_b)
        aux_matrix.at[i, j] = score

In [ ]:
# Set the diagonal to 0 (Same user-user interaction = 0)
np.fill_diagonal(aux_matrix.to_numpy(), 0)

In [ ]:
# Fill adj_matrix with aux_matrix data
for column in aux_matrix.columns:
    for row in aux_matrix.index:

        aux = aux_matrix.at[row, column]
        adj_matrix.at[row, column] = aux

In [ ]:


def graph_metrics(adj_matrix):
  # Build the initial graph from adj_matrix.
  G = nx.from_pandas_adjacency(adj_matrix, create_using=nx.DiGraph)
  d = {'degree' : nx.degree(G),                                   # measure of activity of the node.
       'betweenness_centrality' : nx.betweenness_centrality(G)}   # measure of relative importance of the node.

  metric_df = pd.DataFrame(d)

  #TODO:  filter users
  #       normalise node degree values  https://www.statology.org/normalize-data-between-0-and-1/


  return metric_df

In [ ]:
# TODO: plot graph metric distribution

In [ ]:
def objective(candidate):
  return qmc.discrepancy(candidate, method='L2-star',workers=-1)

In [ ]:
# TODO: 
objective(df)